In [1]:
import pandas as pd
data = pd.read_csv("pima-indians-diabetes.csv",delimiter=",")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
print(data.columns.tolist())

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Class']


In [3]:
import numpy as np
# data['Pregnancies'].replace((0),(np.nan),inplace=True)  不該轉換成 NaN，0 是可能合理的值，有可能沒懷孕過
data['Glucose'].replace((0),(np.nan),inplace=True)
data['BloodPressure'].replace((0),(np.nan),inplace=True)
data['SkinThickness'].replace((0),(np.nan),inplace=True)
data['Insulin'].replace((0),(np.nan),inplace=True)
data['BMI'].replace((0),(np.nan),inplace=True)
# data['DiabetesPedigreeFunction'].replace((0),(np.nan),inplace=True) 一個人家族中有可能完全沒有人有糖尿病
data['Age'].replace((0),(np.nan),inplace=True)
data.head()

C:\Users\617816\AppData\Local\Temp\ipykernel_9792\1006733465.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Glucose'].replace((0),(np.nan),inplace=True)
C:\Users\617816\AppData\Local\Temp\ipykernel_9792\1006733465.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [4]:
data.isna().sum() #check missing values amount

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Class                         0
dtype: int64

In [5]:
cols_to_fill = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in cols_to_fill:
    data[col] = data[col].fillna(data.groupby('Class')[col].transform('mean').round())

In [6]:
data.isna().sum() #再次確認是否有遺漏miss value

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Class                       0
dtype: int64

In [7]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148.0,72.0,35.0,207.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,130.0,26.6,0.351,31,0
2,8,183.0,64.0,33.0,207.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [8]:
# 取出最後 100 筆作為預測資料
inference_data = data.tail(100)

# 匯出成 CSV 檔案
inference_data.to_csv('pima-indians-diabetes_tableau.csv', index=False)

In [9]:
Q1 = data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'DiabetesPedigreeFunction']].quantile(0.25)
Q3 = data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'DiabetesPedigreeFunction']].quantile(0.75)
IQR = Q3 - Q1

# 篩選出異常值
outliers = ((data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'DiabetesPedigreeFunction']] < (Q1 - 1.5 * IQR)) |
            (data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'DiabetesPedigreeFunction']] > (Q3 + 1.5 * IQR)))

print(outliers.sum())  # 每個欄位異常值數量

Glucose                      0
BloodPressure               14
SkinThickness               53
Insulin                     26
DiabetesPedigreeFunction    29
dtype: int64


In [10]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

minmax_scaler = MinMaxScaler()
robust_scaler = RobustScaler()

# 用 MinMaxScaler 處理沒有離群值的欄位
data[['Glucose']] = minmax_scaler.fit_transform(data[['Glucose']])

# 用 RobustScaler 處理有離群值的欄位
cols_with_outliers = ['BloodPressure', 'SkinThickness', 'Insulin', 'DiabetesPedigreeFunction']
data[cols_with_outliers] = robust_scaler.fit_transform(data[cols_with_outliers])

data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,0.670968,0.000,0.875,0.900585,33.6,0.665359,50,1
1,1,0.264516,-0.375,0.125,0.000000,26.6,-0.056209,31,0
2,8,0.896774,-0.500,0.625,0.900585,23.3,0.783007,32,1
3,1,0.290323,-0.375,-0.625,-0.421053,28.1,-0.537255,21,0
4,0,0.600000,-2.000,0.875,0.444444,43.1,5.007843,33,1


In [11]:
bins = [-np.inf, 18.5, 25, 30, np.inf]
labels = [0, 1, 2, 3]
# Underweight
# Healthy Weight
# Overweight
# Obesity
data['BMI'] = pd.cut(data['BMI'], bins=bins, labels=labels)

In [12]:
data['BMI'].unique() #查看BMI資料有哪些

[3, 2, 1, 0]
Categories (4, int64): [0 < 1 < 2 < 3]

In [13]:
import pandas as pd

# 定義年齡區間邊界
bins = [10, 20, 30, 40, 50, 60, 70, 80, float('inf')]
labels = [1, 2, 3, 4, 5, 6, 7, 8]

# 套用分箱
data['Age'] = pd.cut(data['Age'], bins=bins, labels=labels)

In [14]:
data['Age'].unique() #查看Age資料有哪些

[4, 3, 2, 5, 6, 7, 8]
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]

In [15]:
data_corr = data.corr()
data_corr['Class'].sort_values(ascending = False)

Class                       1.000000
Glucose                     0.495901
Insulin                     0.411877
SkinThickness               0.311242
BMI                         0.307594
Age                         0.230526
Pregnancies                 0.221898
BloodPressure               0.174175
DiabetesPedigreeFunction    0.173844
Name: Class, dtype: float64

In [16]:
# 分出訓練 + 測試資料（前 668 筆）
train_test_data = data.head(len(data) - 100)
# 分出訓練 + 測試資料（後 100 筆）
inference_data = data.tail(100)

# 定義特徵 X 和目標變數 Y
X = train_test_data.drop(['Class'], axis=1)
Y = train_test_data['Class']


# 建立隨機森林模型並訓練
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, Y)

# 計算特徵的重要性
feature_importance = model.feature_importances_

# 顯示每個特徵的重要性
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# 根據重要性排序
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                    Feature  Importance
4                   Insulin    0.372540
1                   Glucose    0.172805
3             SkinThickness    0.153023
6  DiabetesPedigreeFunction    0.089585
2             BloodPressure    0.066835
0               Pregnancies    0.062168
7                       Age    0.048802
5                       BMI    0.034242


In [17]:
train_test_data.to_csv('pima-indians-diabetes_traintest.csv', index=False, encoding='utf-8-sig')
inference_data.to_csv('pima-indians-diabetes_predict.csv', index=False, encoding='utf-8-sig')

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# 1. 資料切分
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# 2. GridSearch 參數
LR_parameters = [{
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10, 50, 100],
    'solver': ['liblinear']
}]

SVC_parameters = [{
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10, 50, 100]
}]

GNB_parameters = [{}]

# 3. 模型與對應參數設置
models_with_params = [
    ('LogisticRegression', LogisticRegression(max_iter=1000), LR_parameters),
    ('SVC', SVC(), SVC_parameters),
    ('GaussianNB', GaussianNB(), GNB_parameters)
]

# 4. 建立結果記錄表
model_name = []
model_accuracy = []
model_precision = []
model_recall = []
model_f1 = []

# 5. 執行 GridSearchCV 並評估最佳模型
for name, model, params in models_with_params:
    print(f"GridSearch for {name}...")
    grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, Y_train)
    best_model = grid.best_estimator_

    print(f"Best parameters for {name}: {grid.best_params_}")

    y_pred = best_model.predict(X_test)
    model_name.append(name)
    model_accuracy.append(accuracy_score(Y_test, y_pred))
    model_precision.append(precision_score(Y_test, y_pred, zero_division=0))
    model_recall.append(recall_score(Y_test, y_pred, zero_division=0))
    model_f1.append(f1_score(Y_test, y_pred, zero_division=0))

# 6. 輸出表格
grid_model_result = pd.DataFrame({
    'Model': model_name,
    'Accuracy': model_accuracy,
    'Precision': model_precision,
    'Recall': model_recall,
    'F1': model_f1
})
grid_model_result = grid_model_result.sort_values(by='Accuracy', ascending=False)

print(grid_model_result)


GridSearch for LogisticRegression...
Best parameters for LogisticRegression: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
GridSearch for SVC...
Best parameters for SVC: {'C': 10, 'kernel': 'rbf'}
GridSearch for GaussianNB...
Best parameters for GaussianNB: {}
                Model  Accuracy  Precision    Recall        F1
1                 SVC  0.850746   0.777778  0.753846  0.765625
0  LogisticRegression  0.840796   0.761905  0.738462  0.750000
2          GaussianNB  0.800995   0.671233  0.753846  0.710145
